# Class 11: Cryptography with python

## Learning outcomes

At the completion of this unit students should be able to:
1.   Understand the concepts of encoding and decoding a plain text message
2. Apply the `cryptography` module to implement symmetric encryption
3. Apply the `rsa` module to implement asymmetric encryption

## 11.1 Terminology

- **Plain Text**:  is the text which is readable, and which we want to keep as a secret.
- **Cipher Text**: is the message obtained after applying cryptography on plain text.
- **Hasing**: converting data into a unique string. The original data cannot be recovered from the hash string (it leads to data loss).
- **Encryption (or encoding)**: is the process of converting plain text to cipher text.
- **Decryption (or decodinig)**: is the process of converting cipher text to plain text.
- **Symmetric encryption**: is when a plain text can be encrypted and decrypted using the same key.
- **DES (Data Encryption Standard)**: is a symmetric block cipher standardized by NIST limited to 56 bits. It is outdated, and superceded by AES because it is not secure.
- **AES (Advanced Encryption Standard)**: is a symmetric block cipher standardized by NIST. AES is fast, cryptographically strong, and secure. It is a good default choice for encryption. For more information: https://en.wikipedia.org/wiki/Advanced_Encryption_Standard.
- **KDF (Key Derivation Function)**: A cryptographic hash function that derives one or more secret keys from a password. For more information: https://en.wikipedia.org/wiki/Key_derivation_function.
- **Fernet**: is an implementation of symmetric cryptography. For more information: https://cryptography.io/en/latest/fernet/.


Here we will focus on the Fernet class for symmetric cryptography.

## 11.2 Python string prefixes: row and byte

A python string literal can be prefixed by a letter that indicates the type of characters in that string. String prefixes are documented here: https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals. Will briefly discuss it in today's class.

There are two types of string literals (or string values) in python: row string literals, which are the ones we have been dealing with so far, and byte string literals.

Bytes string literals are always prefixed with 'b' or 'B'. For example, this is a byte string literal: `b"Hi there!"`, whereas this is a row string literatal: `"Hi there!"`.

A byte string literal is an instance of the *bytes* type instead of the *str* type. Let's check that using the `type()` function.

In [31]:
a = b"Hi there!"
print(type(a))

a = r"Hi there!"
print(type(a))

a = "Hi there!"
print(type(a))

<class 'bytes'>
<class 'str'>
<class 'str'>


In [32]:
b = b'Some text string'
print(b)
print('Decode byte to str: ',b.decode('utf-8'))

r = r'Some text string'
print(r)
print('Encode str to byte', r.encode('utf-8'))



b'Some text string'
Decode byte to str:  Some text string
Some text string
Encode str to byte b'Some text string'


They may only contain ASCII characters; bytes with a numeric value of 128 or greater must be expressed with escapes.

### Generation of random numbers and bytes

Random number and string generation is very impotant in cryptography because it enables key generators to have unexpected (random) features which cannot be prediced by adversaries. To generate a random number in python, we use the `random()` function.

In [62]:
import random

random.randint(3,1000)

for i in range(10):
  print(random.random())

0.5584869595196068
0.7471074138168893
0.917805989397803
0.8237954130958162
0.9625718572476318
0.6624389929548126
0.21116674835578264
0.2289296075151711
0.032298020440548636
0.20924386482426516


In [68]:
import numpy

numpy.random.seed(10)
print(numpy.random.random())

for _ in range(10):
    print(numpy.random.random())

0.771320643266746
0.0207519493594015
0.6336482349262754
0.7488038825386119
0.4985070123025904
0.22479664553084766
0.19806286475962398
0.7605307121989587
0.16911083656253545
0.08833981417401027
0.6853598183677972


These numbers are random numbers in the range from 0 to 1. We can also generate a random byte string using the `urandom()` function in the `os` module. Note, in the example below, that `urandom()` receive an integer parameter which is the size of the random byte string. The difference between these two is that `urandom()` does not depend on a seed, but it instead generates randomness from various sources. Hence, it can be considered more random than `random()`.


In [45]:
import os

a = os.urandom(1)
print(a)


b'\x02'


## 11.3 Hashing

Hashing can be performed using the built-in function `hash()`. You can only hash immutable objects like numbers, strings and tuples.

In [75]:
print(hash('abC'))
print(len(str(hash('abC'))))

3974803468147435309
19


Hashing mutable objects will give an error.

In [71]:
hash([1,2,3])

TypeError: unhashable type: 'list'

Another library to use for hashing is `hashlib`.

In [78]:
import hashlib

print(hashlib.md5(b'abc').hexdigest())

900150983cd24fb0d6963f7d28e17f72


You can make your own python object hashable by overriding the `__hash__()` class method.

In [ ]:
class Employee:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    def __hash__(self):      
        return hash((self.first_name, self.last_name))

e = Employee('A','C')
print(hash(e))

162886057105



## 11.4 Symmetric encryption: `Fernet`

The Fernet class, which is part of the `cryptography` package, offers symmetric encryption based on the AES. Fernet is a standard for encrypting buffers in a way that follows best-practices cryptography. It is not suitable for very big files.

In symmetric, or secret key, cryptography: the same key is used for encryption and decryption, and therefore must be kept safe.



In [92]:
#pip3 install cryptography
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print('The key: ', key)
print('Creating a Fernet object with the key.')
fkey = Fernet(key)
cipher_text = fkey.encrypt(b"I am working from home.")
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

The key:  b'dUVcxNKjzkhGSeiVSMgp9Zk7vMrenp8bUd5PQ2mZ8ZI='
Creating a Fernet object with the key.
Cipher:  b'gAAAAABoJGZM7DqbHqc7iEX8SH5fq2IeOAlQkYs0gcFKZltLf1uzcVdX4QCP7tF4v6Oml9CR6UO6gBDl7KA3KW0I4IhQANP09oXGMEyen9DvpjXcKKvE1dQ='
Plain:  b'I am working from home.'


You will get slightly different values if you encrypt on your machine.

Note that this only encrypts and decrypts byte strings. In order to encrypt and decrypt text strings, they will need to be encoded and decoded, usually with UTF-8.


In [19]:
from cryptography.fernet import Fernet

st = "I am working from home."
st=st.encode('utf-8')

key = Fernet.generate_key()
fkey = Fernet(key)
print('The key object: ', fkey)
cipher_text = fkey.encrypt(st)
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

The key object:  <cryptography.fernet.Fernet object at 0x0000025EB5A749D0>
Cipher:  b'gAAAAABoJCVuw4tWMwlnKyJ2CzQxoleF3JlT74vsv9ixgGW5JM9qJgcA7beQfIZ4tACV6rQp6goNt_otp0SzFwsewZi0V2qZtAMNSzwzjgREM9nFFGPElbM='
Plain:  b'I am working from home.'


### Fernet key from a password

You can create a new key based on a password. There are many differernt `cryptography` classes that can *derive* a key based on a password. Here will use one of them, `PBKDF2HMAC`.

In [20]:
import base64
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
password = b"my complicated password"

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=os.urandom(10),
    iterations=100000,
    backend=default_backend()
)
pre_key = kdf.derive(password)
print(pre_key)

key = base64.urlsafe_b64encode(pre_key)
print(key)
fkey = Fernet(key)
print('The key object: ', fkey)
cipher_text = fkey.encrypt(st)
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

b'9\x91`\xcc\xd0\xdd\xfd\x86\x85\x88{\x87\xd3L\x10\x9e0\xf50\xf5H\xf4*\xdd\x90-1\x17\xca]\x9f\xf5'
b'OZFgzNDd_YaFiHuH00wQnjD1MPVI9CrdkC0xF8pdn_U='
The key object:  <cryptography.fernet.Fernet object at 0x0000025EB5B5E290>
Cipher:  b'gAAAAABoJCWKcF9Fcb_sse542D11LbycdaSd1kfJWYcT-DcdCXFrBS0o2bLthohc2vgQgk2DPhpv1aqbGx_SYGZ5fZAeBUR0uSV5_4i-eKWWXBGMKfmsVGY='
Plain:  b'I am working from home.'


## 11.5 Asymmetric encryption

The `rsa` library is an implementation of asymmetric encryption. Here is an example:

In [93]:
import rsa

public_key, private_key = rsa.newkeys(300,accurate=False)
print(public_key)

message = b"This is a trade secret."

crypt = rsa.encrypt(message, public_key)

print(crypt)

decrypt = rsa.decrypt(crypt, private_key)

print(decrypt.decode())

PublicKey(1148349299115540083601576465465899803471056215051743011378578638167793456353452779714397843, 65537)
b'\x04\xca~\x9e\xb1R\x02\xd2\xc7FU\r\xdb\xc6}\x83\xbb\xd4\xef;\xf5!\xf9\x84L\x7f\xc9Y uCe\x80\xd6^\x19N\x00'
This is a trade secret.


`nbits` determines the cryptographic strength of the key and size of the message string.